## 已知n,e,c,yafu分解n后，求明文(m)
通过yafu分解N，得到p，q

yafu-x64.exe factor(108539847268573990275234024354672437246525085076605516960320005722741589898641)

In [3]:
#已知n,e,c,yafu分解n后，求明文(m)
import math
n = 0xEFF76062D52A7D4927DF587FBBC293CF9A622F56E854055D6A30DCF77C9B7591
#通过yafu分解N，得到p，q
#yafu-x64.exe factor(108539847268573990275234024354672437246525085076605516960320005722741589898641)
q = 333360321402603178263879595968004169219
p = 325593180411801742356727264127253758939
e=0x10001
c=0x127922e5f701afb823a407af5f9afcc750d8053fbe1206ce6afa240bd87748ef
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
          raise Exception('modular inverse does not exist')
    else:
          return x % m
d=modinv(e,(p-1)*(q-1))
print(d)
#m是明文
m=pow(c,d,n)
bytes.fromhex(str(hex(m))[2:])

14387073763249801692493674864107900306007889970401611155360615803588944959757


b'flag{18400f2b264e100c}'

## 直接或间接给出p ,q ,e ,c这四个参数，求明文m
把p,q的位置改成题目中给的值，并且按照脚本的提示输入密文c ，即可求得答案。

In [30]:
# -*- coding: utf-8 -*-
#已知p,q,e,c求明文m
import gmpy2
p = 289540461376837531747468286266019261659
q = 306774653454153140532319815768090345109
n = p*q
e = 65537
print("密文c：")
c = int(input())
# d=invert(e,φ(n))，可见invert()函数解决 ed ≡ 1 (mod φ(n)) 问题
d=gmpy2.invert(e,(p-1)*(q-1))
print("私钥d：",d)
key=pow(c, d, n)
print("明文：",bytes.fromhex(str(hex(key))[2:]))

密文c：
306774653454153140532319815768090345109
私钥d： 34151379451095426498039388462988737764164034221083995907950386968238819049329
明文： b'\x19~\xdb\xad8\xdd\x15\xce\xd9\xa8\x05.)\xa3\xe5\x99\xdb\x92"\x90Y\x80b\x83r(\r\xc1\x17\x85\xe7\xa9'


## 得到了n, e, c,但n的分解非常困难
这时候可以考虑用wiener-attack方法求出d和m，wiener-attack脚本在github有，这里列一下利用脚本：

In [ ]:
#!/usr/bin/python
import ContinuedFractions,Arithmetic
import time
import sys
import binascii
sys.setrecursionlimit(100000)
# modulus from the RSA public key
n=input(“input n:”)
# exponent from the RSA public key
e=input(“input e:”)
def hack_RSA(e,n):
    print “Performing Wiener’s attack. Don’t Laugh…”
    time.sleep(1)
    frac = ContinuedFractions.rational_to_contfrac(e, n)
    convergents = ContinuedFractions.convergents_from_contfrac(frac)
    for (k,d) in convergents:
    #check if d is actually the key
    if k!=0 and (e*d-1)%k == 0:
    phi = (e*d-1)//k
    s = n – phi + 1
    # check if the equation x^2 – s*x + n = 0
    # has integer roots
    discr = s*s – 4*n
    if(discr>=0):
        t = Arithmetic.is_perfect_square(discr)
    if t!=-1 and (s+t)%2==0:
        return d
hacked_d = hack_RSA(e, n)
print “d=” + str(hacked_d)
d=hacked_d
c=input(“input c:”)
m = hex(pow(c, d, n)).rstrip(“L”)
m1=pow(c, d, n)
print binascii.unhexlify(m[2:])
print(“%0512x” %m1).decode(“hex”)

## 题目一：
182812482972168423884795132699225934365072979206288632257180603530046820174392675977209758378734399146216742345585898385168866887000708558119959898992294085847474548306743585711154035585848291290988967352517174312220756638881837930962458861193652684492265539096477345065113556380573776423787885892688197584678128636231428194711357642971544417113415626331810909274966752557628893585198569815939514862013512237657828262360291726912615575646318630641527418369988268899879152029186728850816178597399494254385226049249357897840618728804680238123954207656671747782543031545429711152272581734051959578453680011676521727918037340906791388178004979453256050227967701258768070039292546964652071924183467364467145178290753361477912582242961929982420950384199259355122986865808523351306098081481072454093823090
438980397031315392229453908048509540832246041631432878509579665664182747463100230160823865621798053164989325086085003940181731721089701380743698761443812523024144817205902380903062054138730658451286904347536210833160924917347633148983052015550354913154312162901555870494273903714349869746793861874257201085777893961715468950661641778512110325457371446203379767458862059193946434683324578530163650541637261158037041205642428802942295011562277084687025213626698849526240663754073508102229066475773893638716845176469070938803298515155140240970836387785401085919369741520890271902332951669953411373633688944162470994856654604872287103746922041844065053274059990595496159866206551119361036237431289830985174384522423364811997241255005514248198447925396378192915553898993758660041223393168707380580012437
```python
from flag import FLAG
from Crypto.Util.number import *
import gmpy2
import random

while True:
    p = int(gmpy2.next_prime(random.randint(10**399, 10**400-1)))
    q = int(str(p)[200:]+str(p)[:200])
    if gmpy2.is_prime(q):
        break

m = bytes_to_long(FLAG)
n = p*q
e = 65537
c = pow(m,e,n)

with open("enc","wb") as f:
    f.write(str(c))
    f.write("\n")
    f.write(str(n))
```
### 题解

In [21]:
import gmpy2
c = 182812482972168423884795132699225934365072979206288632257180603530046820174392675977209758378734399146216742345585898385168866887000708558119959898992294085847474548306743585711154035585848291290988967352517174312220756638881837930962458861193652684492265539096477345065113556380573776423787885892688197584678128636231428194711357642971544417113415626331810909274966752557628893585198569815939514862013512237657828262360291726912615575646318630641527418369988268899879152029186728850816178597399494254385226049249357897840618728804680238123954207656671747782543031545429711152272581734051959578453680011676521727918037340906791388178004979453256050227967701258768070039292546964652071924183467364467145178290753361477912582242961929982420950384199259355122986865808523351306098081481072454093823090
n = 438980397031315392229453908048509540832246041631432878509579665664182747463100230160823865621798053164989325086085003940181731721089701380743698761443812523024144817205902380903062054138730658451286904347536210833160924917347633148983052015550354913154312162901555870494273903714349869746793861874257201085777893961715468950661641778512110325457371446203379767458862059193946434683324578530163650541637261158037041205642428802942295011562277084687025213626698849526240663754073508102229066475773893638716845176469070938803298515155140240970836387785401085919369741520890271902332951669953411373633688944162470994856654604872287103746922041844065053274059990595496159866206551119361036237431289830985174384522423364811997241255005514248198447925396378192915553898993758660041223393168707380580012437
e = 65537

# 通过拼接求p，q
ab_high = str(n // 10**600)
ab_low = str(n)[-200:]
ab = int(ab_high+ab_low)
print(ab)
a2b2 = (n - ab*(10**400+1))//10**200
print(a2b2)
a_p_b = gmpy2.iroot(a2b2+2*ab, 2)[0]
a_m_b = gmpy2.iroot(a2b2-2*ab, 2)[0]
a = (a_p_b+a_m_b)//2
b = (a_p_b-a_m_b)//2
p = a*10**200+b
q = b*10**200+a
print(p)
print(q)
print('p*q == n: {}'.format(p*q == n))

# 通过pq，还原密文
phi = (p-1)*(q-1)
d = gmpy2.invert(e,phi)
m = pow(c,d,p*q)
bytes.fromhex(str(hex(m))[2:])

43898039703131539222945390804850954083224604163143287850957966566418274746310023016082386562179805316498932508608500394018173172108970138074369876144381252302414481720590238090306205413873065845128688944162470994856654604872287103746922041844065053274059990595496159866206551119361036237431289830985174384522423364811997241255005514248198447925396378192915553898993758660041223393168707380580012437
149059115008845943788686247621115138315971097043804137122569465942718322083925207395073724809635644054572405536603135954965381115866360301258462034076642742242284689244725935342007516375047243635261715933631183339096664194834121374440574790524805238819257757408829598223890077634850861466799135940883222179126678605705447851253430461701028964665116410198336169552598003000338120267462560803455285050
116085856120094667394963460441002535135432659349302581401408741027178854111922032225176817246081219154381041091553471586184568580921485762558578634559416585952247012244124802464275373874552529551545530

b'CMISCCTF{easy_math_game_hhhhhhh}'

In [ ]:
import gmpy2
p = 3487583947589437589237958723892346254777
q = 8767867843568934765983476584376578389
e = 65537
d = gmpy2.invert(e,(p-1)*(q-1))
print(d)

In [ ]:
from libnum import gcd
n1=55
n2=77
print ("p -> {}".format(gcd(n1, n2)))
print ("q1 -> {}".format(n1 / gcd(n1, n2)))
print ("q2 -> {}".format(n2 / gcd(n1, n2)))

In [ ]:
hex(38321129010629359589718877669457409338494938118120317)

In [ ]:
0xEFF76062D52A7D4927DF587FBBC293CF9A622F56E854055D6A30DCF77C9B7591

In [ ]:
bytes.fromhex('666c61677b313834303066326232363465313030637d')